In [ ]:
#@title Prepare the running enviroment
%cd /content
!git clone https://github.com/joeyz0z/ConZIC.git
%cd /content/ConZIC
!pip install -r requirements.txt
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('sentiwordnet')

/content
Cloning into 'ConZIC'...
remote: Enumerating objects: 425, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 425 (delta 32), reused 27 (delta 23), pack-reused 371 (from 1)
Receiving objects: 100% (425/425), 11.86 MiB | 19.90 MiB/s, done.
Resolving deltas: 100% (167/167), done.
/content/ConZIC


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.


True

In [ ]:
#@title Upload your image or use our examples
upload_your_image = True # @param {type:"boolean"}

In [ ]:
import os
#@title Select examples
example_name = 'girl.jpg' # @param ['cat.png', 'girl.jpg', 'Gosh.jpeg', 'horse.png']
example_img_path = '/content/ConZIC/examples/girl.jpg'
x= 'string'
print(x)
print(example_img_path)

string
/content/ConZIC/examples/girl.jpg


In [ ]:
#@title Upload image

import os
from google.colab import files
!mkdir your_uploaded_image
%cd /content/ConZIC/your_uploaded_image
uploaded = files.upload()

if not uploaded:
  img_name = ''
elif len(uploaded) == 1:
  img_name = list(uploaded.keys())[0]
else:
  raise AssertionError('Please upload one image at a time')

upload_img_path = os.path.join(os.getcwd(), img_name)
print(upload_img_path)
%cd /content/ConZIC


mkdir: cannot create directory ‘your_uploaded_image’: File exists
/content/ConZIC/your_uploaded_image


Saving data1.jpg to data1 (1).jpg
/content/ConZIC/your_uploaded_image/data1 (1).jpg
/content/ConZIC


In [ ]:
# @title Import
import sys
sys.path.append('/content/ConZIC/examples')

from utils import create_logger,set_seed
import os
import time
import argparse
import json
from PIL import Image
import torch
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

from clip.clip import CLIP
from gen_utils import generate_caption
from control_gen_utils import control_generate_caption
from transformers import AutoModelForMaskedLM, AutoTokenizer
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from transformers import AutoTokenizer, AutoModelForSequenceClassification


model_for_translation = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer_for_translation = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

tokenizer_for_detection = AutoTokenizer.from_pretrained('Mike0307/multilingual-e5-language-detection')
model_for_detection = AutoModelForSequenceClassification.from_pretrained('Mike0307/multilingual-e5-language-detection', num_labels=45)



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.67k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [ ]:
# @title Define parameters
def get_args():
    parser = argparse.ArgumentParser()

    parser.add_argument("--seed", type=int, default=42)
    parser.add_argument("--batch_size", type=int, default=1, help = "Only supports batch_size=1 currently.")
    parser.add_argument("--device", type=str,
                        default='cuda',choices=['cuda','cpu'])

    ## Generation and Controllable Type
    parser.add_argument('--run_type',
                        default='caption',
                        nargs='?',
                        choices=['caption', 'controllable'])
    parser.add_argument('--prompt',
                        default='Image of a',type=str)
    parser.add_argument('--order',
                        default='shuffle',
                        nargs='?',
                        choices=['sequential', 'shuffle', 'span', 'random','parallel'],
                        help="Generation order of text")
    parser.add_argument('--control_type',
                        default='sentiment',
                        nargs='?',
                        choices=["sentiment","pos"],
                        help="which controllable task to conduct")
    parser.add_argument('--pos_type', type=list,
                        default=[['DET'], ['ADJ','NOUN'], ['NOUN'],
                                 ['VERB'], ['VERB'],['ADV'], ['ADP'],
                                 ['DET','NOUN'], ['NOUN'], ['NOUN','.'],
                                 ['.','NOUN'],['.','NOUN']],
                        help="predefined part-of-speech templete")
    parser.add_argument('--sentiment_type',
                        default="positive",
                        nargs='?',
                        choices=["positive", "negative"])
    parser.add_argument('--samples_num',
                        default=2,type=int)

    ## Hyperparameters
    parser.add_argument("--sentence_len", type=int, default=10)
    parser.add_argument("--candidate_k", type=int, default=200)
    parser.add_argument("--alpha", type=float, default=0.02, help="weight for fluency")
    parser.add_argument("--beta", type=float, default=2.0, help="weight for image-matching degree")
    parser.add_argument("--gamma", type=float, default=5.0, help="weight for controllable degree")
    parser.add_argument("--lm_temperature", type=float, default=0.1)
    parser.add_argument("--num_iterations", type=int, default=10, help="predefined iterations for Gibbs Sampling")

    ## Models and Paths
    parser.add_argument("--lm_model", type=str, default='bert-base-uncased',
                        help="Path to language model") # bert,roberta
    parser.add_argument("--match_model", type=str, default='openai/clip-vit-base-patch32',
                        help="Path to Image-Text model")  # clip,align
    parser.add_argument("--caption_img_path", type=str, default='./examples/girl.jpg',
                        help="file path of the image for captioning")
    parser.add_argument("--stop_words_path", type=str, default='stop_words.txt',
                        help="Path to stop_words.txt")
    parser.add_argument("--add_extra_stopwords", type=list, default=[],
                        help="you can add some extra stop words")

    args = parser.parse_args(args=[])
    return args

In [ ]:
# @title Image captioning
def run_caption(args, image_path, lm_model, lm_tokenizer, clip, token_mask, logger):
    FinalCaptionList = []
    BestCaptionList = []
    logger.info(f"Processing: {image_path}")
    image_instance = Image.open(image_path).convert("RGB")
    # img_name = [image_path.spilt("/")[-1]]
    for sample_id in range(args.samples_num):
        logger.info(f"Sample {sample_id}: ")
        gen_texts, clip_scores = generate_caption(image_path,lm_model, clip, lm_tokenizer, image_instance, token_mask, logger,
                                  prompt=args.prompt, batch_size=args.batch_size, max_len=args.sentence_len,
                                  top_k=args.candidate_k, temperature=args.lm_temperature,
                                  max_iter=args.num_iterations,alpha=args.alpha,beta=args.beta,
                                  generate_order = args.order)
        FinalCaptionStr = "Sample {}: ".format(sample_id + 1) + str(gen_texts[-2])
        BestCaptionStr = "Sample {}: ".format(sample_id + 1) + str(gen_texts[-1])
        FinalCaptionList.append(FinalCaptionStr)
        BestCaptionList.append(BestCaptionStr)
    return FinalCaptionList, BestCaptionList
def run_control(run_type, args, image_path, lm_model, lm_tokenizer, clip, token_mask, logger):
    FinalCaptionList = []
    BestCaptionList = []
    logger.info(f"Processing: {image_path}")
    image_instance = Image.open(image_path).convert("RGB")
    for sample_id in range(args.samples_num):
        logger.info(f"Sample {sample_id}: ")
        gen_texts, clip_scores = control_generate_caption(image_path,lm_model, clip, lm_tokenizer, image_instance, token_mask, logger,
                                  prompt=args.prompt, batch_size=args.batch_size, max_len=args.sentence_len,
                                  top_k=args.candidate_k, temperature=args.lm_temperature,
                                  max_iter=args.num_iterations, alpha=args.alpha,
                                  beta=args.beta, gamma=args.gamma,
                                  ctl_type = args.control_type, style_type=args.sentiment_type,pos_type=args.pos_type, generate_order=args.order)
        FinalCaptionStr = "Sample {}: ".format(sample_id + 1) + str(gen_texts[-2])
        BestCaptionStr = "Sample {}: ".format(sample_id + 1) + str(gen_texts[-1])
        FinalCaptionList.append(FinalCaptionStr)
        BestCaptionList.append(BestCaptionStr)
    return FinalCaptionList, BestCaptionList

In [ ]:
# @title GPU/CPU
is_gpu = True #@param {type:"boolean"}



*   RunType: Select RunType equal to "contrallable" to control text generation.
*   ControlType: Control text by sentiment or part of speech.
*   SentimentType: Control sentiment: positive or negative
*   Order: Generation order of text
*   Alpha: Weight for fluency; Choose between 0 and 1
*   Beta: Weight for image-matching degree; Choose between 1 and 5
*   Gamma: Weight for controllable degree; Choose between 1 and 10
*   SampleNum: Number of runs; Choose between 1 and 5
*   Length: Sentence length; Choose between 5 and 10
*   NumIterations: Iterations for Gibbs Sampling; Choose between 1 and 15

In [ ]:
# @title Select types and parameters
args = get_args()

RunType = 'caption' # @param ['caption', 'controllable']
ControlType = 'sentiment' # @param ["sentiment","pos"]
SentimentType = 'positive' # @param ["positive", "negative"]
Order = 'shuffle' # @param ['sequential', 'shuffle', 'random']
Alpha = 0.02 # @param {type:"slider", min:0, max:1, step:0.01}
Beta = 2 # @param {"type":"slider","min":2,"max":5,"step":0.5}
Gamma = 4 # @param {type:"slider", min:1, max:10, step:0.5}
SamplesNum = 2 # @param {type:"slider", min:1, max:5, step:1}
Length = 10 # @param {type:"slider", min:5, max:15,  step:1}
NumIterations = 10 # @param {type:"slider", min:1, max:15, step:1}

args.sentence_len = Length
args.run_type = RunType
args.control_type = ControlType
args.sentiment_type = SentimentType
args.alpha = Alpha
args.beta = Beta
args.gamma = Gamma
args.samples_num = SamplesNum
args.order = Order
args.num_iterations = NumIterations
args.caption_img_path = example_img_path
args.device = "cuda"
set_seed(args.seed)

In [ ]:
# @title Create logger

run_type = "caption" if args.run_type=="caption" else args.control_type
if run_type=="sentiment":
    run_type = args.sentiment_type

if os.path.exists("logger")== False:
    os.mkdir("logger")
logger = create_logger(
    "logger",'demo_{}_{}_len{}_topk{}_alpha{}_beta{}_gamma{}_lmtemp{}_{}.log'.format(
    run_type, args.order, args.sentence_len,
    args.candidate_k, args.alpha, args.beta,args.gamma, args.lm_temperature,
    time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())))

logger.info(f"Generating order:{args.order}")
logger.info(f"Run type:{run_type}")
logger.info(args)

Generating order:shuffle
INFO:ConZIC:Generating order:shuffle
Run type:caption
INFO:ConZIC:Run type:caption
Namespace(seed=42, batch_size=1, device='cuda', run_type='caption', prompt='Image of a', order='shuffle', control_type='sentiment', pos_type=[['DET'], ['ADJ', 'NOUN'], ['NOUN'], ['VERB'], ['VERB'], ['ADV'], ['ADP'], ['DET', 'NOUN'], ['NOUN'], ['NOUN', '.'], ['.', 'NOUN'], ['.', 'NOUN']], sentiment_type='positive', samples_num=2, sentence_len=10, candidate_k=200, alpha=0.02, beta=2, gamma=4, lm_temperature=0.1, num_iterations=10, lm_model='bert-base-uncased', match_model='openai/clip-vit-base-patch32', caption_img_path='/content/ConZIC/examples/girl.jpg', stop_words_path='stop_words.txt', add_extra_stopwords=[])
INFO:ConZIC:Namespace(seed=42, batch_size=1, device='cuda', run_type='caption', prompt='Image of a', order='shuffle', control_type='sentiment', pos_type=[['DET'], ['ADJ', 'NOUN'], ['NOUN'], ['VERB'], ['VERB'], ['ADV'], ['ADP'], ['DET', 'NOUN'], ['NOUN'], ['NOUN', '.'], ['.

In [ ]:
# @title Load pre-trained model (weights)
lm_model = AutoModelForMaskedLM.from_pretrained(args.lm_model)
lm_tokenizer = AutoTokenizer.from_pretrained(args.lm_model)
lm_model.eval()
clip = CLIP(args.match_model)
clip.eval()

lm_model = lm_model.to(args.device)
clip = clip.to(args.device)

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/config.json HTTP/11" 200 0
DEBUG:filelock:Attempting to acquire lock 140484388047312 on /root/.cache/huggingface/hub/.locks/models--bert-base-uncased/45a2321a7ecfdaaf60a6c1fd7f5463994cc8907d.lock
DEBUG:filelock:Lock 140484388047312 acquired on /root/.cache/huggingface/hub/.locks/models--bert-base-uncased/45a2321a7ecfdaaf60a6c1fd7f5463994cc8907d.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /bert-base-uncased/resolve/main/config.json HTTP/11" 200 570


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140484388047312 on /root/.cache/huggingface/hub/.locks/models--bert-base-uncased/45a2321a7ecfdaaf60a6c1fd7f5463994cc8907d.lock
DEBUG:filelock:Lock 140484388047312 released on /root/.cache/huggingface/hub/.locks/models--bert-base-uncased/45a2321a7ecfdaaf60a6c1fd7f5463994cc8907d.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/adapter_config.json HTTP/11" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/model.safetensors HTTP/11" 302 0
DEBUG:filelock:Attempting to acquire lock 140484309304656 on /root/.cache/huggingface/hub/.locks/models--bert-base-uncased/68d45e234eb4a928074dfd868cead0219ab85354cc53d20e772753c6bb9169d3.lock
DEBUG:filelock:Lock 140484309304656 acquired on /root/.cache/huggingface/hub/.locks/models--bert-base-uncased/68d45e234eb4a928074dfd868cead0219ab85354cc53d20e772753c6bb9169d3.lock
DEBUG:urllib3.connectionpool:Resetting dropped

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140484309304656 on /root/.cache/huggingface/hub/.locks/models--bert-base-uncased/68d45e234eb4a928074dfd868cead0219ab85354cc53d20e772753c6bb9169d3.lock
DEBUG:filelock:Lock 140484309304656 released on /root/.cache/huggingface/hub/.locks/models--bert-base-uncased/68d45e234eb4a928074dfd868cead0219ab85354cc53d20e772753c6bb9169d3.lock
BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that 

...

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140484345789360 on /root/.cache/huggingface/hub/.locks/models--openai--clip-vit-base-patch32/9bfb42aa97dcd61e89f279ccaee988bccb4fabae.lock
DEBUG:filelock:Lock 140484345789360 released on /root/.cache/huggingface/hub/.locks/models--openai--clip-vit-base-patch32/9bfb42aa97dcd61e89f279ccaee988bccb4fabae.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /openai/clip-vit-base-patch32/resolve/main/processor_config.json HTTP/11" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /openai/clip-vit-base-patch32/resolve/main/chat_template.json HTTP/11" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /openai/clip-vit-base-patch32/resolve/main/tokenizer_config.json HTTP/11" 200 0


CLIP model initialized.


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/openai/clip-vit-base-patch32/commits/main HTTP/11" 200 4272
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/openai/clip-vit-base-patch32/discussions?p=0 HTTP/11" 200 9799
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/openai/clip-vit-base-patch32/commits/refs%2Fpr%2F21 HTTP/11" 200 5237
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /openai/clip-vit-base-patch32/resolve/refs%2Fpr%2F21/model.safetensors.index.json HTTP/11" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /openai/clip-vit-base-patch32/resolve/refs%2Fpr%2F21/model.safetensors HTTP/11" 302 0
DEBUG:filelock:Attempting to acquire lock 140489473358096 on /root/.cache/huggingface/hub/.locks/models--openai--clip-vit-base-patch32/99d28a652e6ec46629ab7047a0ac82c69b1fe11e0ce672c43af65d3a9a3fc05d.lock
DEBUG:filelock:Lock 140489473358096 acquired on /root/.cache/huggingface/hub

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [ ]:
#@title Remove stop words, token mask
with open(args.stop_words_path,'r',encoding='utf-8') as stop_words_file:
    stop_words = stop_words_file.readlines()
    stop_words_ = [stop_word.rstrip('\n') for stop_word in stop_words]
    stop_words_ += args.add_extra_stopwords
    stop_ids = lm_tokenizer.convert_tokens_to_ids(stop_words_)
    token_mask = torch.ones((1,lm_tokenizer.vocab_size))
    for stop_id in stop_ids:
        token_mask[0,stop_id]=0
    token_mask = token_mask.to(args.device)

In [ ]:
# @title Run
img_path = upload_img_path if upload_your_image else example_img_path
if args.run_type == 'caption':
    FinalCaption, BestCaption = run_caption(args, img_path, lm_model, lm_tokenizer, clip, token_mask, logger)
elif args.run_type == 'controllable':
    FinalCaption, BestCaption = run_control(run_type, args, img_path, lm_model, lm_tokenizer, clip, token_mask, logger)
else:
    raise Exception('run_type must be caption or controllable!')

Processing: /content/ConZIC/examples/girl.jpg
INFO:ConZIC:Processing: /content/ConZIC/examples/girl.jpg
Sample 0: 
INFO:ConZIC:Sample 0: 


Cuda is available.
Device is 0


Order_list:[7, 3, 2, 8, 5, 6, 9, 4, 0, 1]
INFO:ConZIC:Order_list:[7, 3, 2, 8, 5, 6, 9, 4, 0, 1]
iter 1, The 1-th image: /,clip score 0.360: [CLS] image of a romantic landscape drowning her underwater amidst her falling sleep. [SEP]
INFO:ConZIC:iter 1, The 1-th image: /,clip score 0.360: [CLS] image of a romantic landscape drowning her underwater amidst her falling sleep. [SEP]
iter 2, The 1-th image: /,clip score 0.355: [CLS] image of a water painter draw melody in amidst her deep sleep. [SEP]
INFO:ConZIC:iter 2, The 1-th image: /,clip score 0.355: [CLS] image of a water painter draw melody in amidst her deep sleep. [SEP]
iter 3, The 1-th image: /,clip score 0.366: [CLS] image of a tall blue ashren lying crying her watery sleep. [SEP]
INFO:ConZIC:iter 3, The 1-th image: /,clip score 0.366: [CLS] image of a tall blue ashren lying crying her watery sleep. [SEP]
iter 4, The 1-th image: /,clip score 0.372: [CLS] image of a floating blue camelli princess crying her watery breaths. [SEP]
INF

In [ ]:
# @title Output
Image.open(img_path).show()
print("Final Caption\n")
for i in range(len(FinalCaption)):
    print(f"{FinalCaption[i]}\n")
print("Best Caption\n")
for i in range(len(BestCaption)):
    print(f"{BestCaption[i]}\n")


caption_1 = FinalCaption[0]
caption_2 = FinalCaption[0][1]


Final Caption

Sample 1: ['image of a girl representing camelot lily drifting and floating softly.']

Sample 2: ['image of a girl sleeping by blue rememberedrin all within water.']

Best Caption

Sample 1: ['image of a transparent blue camelot girl drowning and drifting asleep.']

Sample 2: ['image of a girl drawn color painted aela sleeping amidst water.']



In [ ]:
caption_1 = FinalCaption[1]

print(caption_1)

Sample 2: ['image of a girl sleeping by blue rememberedrin all within water.']


# Taking Custom language as input

1.   List item
2.   List item





In [ ]:
text = input("Enter the text example in the following languages: Arabic Czech German English Spanish Estonian Finnish French Gujarati Hindi Italian Japanese Kazakh Korean Lithuanian Latvian Burmese Nepali Dutch Romanian Russian Sinhala Turkish Vietnamese Chinese ")



Enter the text example in the following languages: Arabic Czech German English Spanish Estonian Finnish French Gujarati Hindi Italian Japanese Kazakh Korean Lithuanian Latvian Burmese Nepali Dutch Romanian Russian Sinhala Turkish Vietnamese Chinese Hola mi nombre es Viswa


#Detecting the language and generation of language code for translation

In [ ]:
languages = [
    "Arabic", "Basque", "Breton", "Catalan", "Chinese_China", "Chinese_Hongkong",
    "Chinese_Taiwan", "Chuvash", "Czech", "Dhivehi", "Dutch", "English",
    "Esperanto", "Estonian", "French", "Frisian", "Georgian", "German", "Greek",
    "Hakha_Chin", "Indonesian", "Interlingua", "Italian", "Japanese", "Kabyle",
    "Kinyarwanda", "Kyrgyz", "Latvian", "Maltese", "Mongolian", "Persian", "Polish",
    "Portuguese", "Romanian", "Romansh_Sursilvan", "Russian", "Sakha", "Slovenian",
    "Spanish", "Swedish", "Tamil", "Tatar", "Turkish", "Ukranian", "Welsh"
]

def predict(text, model, tokenizer, device = torch.device('cpu')):
    model.to(device)
    model.eval()
    tokenized = tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors="pt")
    input_ids = tokenized['input_ids']
    attention_mask = tokenized['attention_mask']
    with torch.no_grad():
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    return probabilities

def get_topk(probabilities, languages, k=3):
    topk_prob, topk_indices = torch.topk(probabilities, k)
    topk_prob = topk_prob.cpu().numpy()[0].tolist()
    topk_indices = topk_indices.cpu().numpy()[0].tolist()
    topk_labels = [languages[index] for index in topk_indices]
    return topk_prob, topk_labels

probabilities = predict(text, model_for_detection, tokenizer_for_detection)
topk_prob, topk_labels = get_topk(probabilities, languages)

# Define a dictionary for language and codes
language_codes = {
    "Arabic": "ar_AR",
    "Czech": "cs_CZ",
    "German": "de_DE",
    "English": "en_XX",
    "Spanish": "es_XX",
    "Estonian": "et_EE",
    "Finnish": "fi_FI",
    "French": "fr_XX",
    "Gujarati": "gu_IN",
    "Hindi": "hi_IN",
    "Italian": "it_IT",
    "Japanese": "ja_XX",
    "Kazakh": "kk_KZ",
    "Korean": "ko_KR",
    "Lithuanian": "lt_LT",
    "Latvian": "lv_LV",
    "Burmese": "my_MM",
    "Nepali": "ne_NP",
    "Dutch": "nl_XX",
    "Romanian": "ro_RO",
    "Russian": "ru_RU",
    "Sinhala": "si_LK",
    "Turkish": "tr_TR",
    "Vietnamese": "vi_VN",
    "Chinese (Simplified)": "zh_CN"
}

# Function to get the language code
def get_language_code(language_name: str) -> str:
    # Check if the language exists in the dictionary
    if language_name in language_codes:
        return language_codes[language_name]
    else:
        return "Language not found."

# Example usage
language_name = topk_labels[0]
language_code = get_language_code(language_name)
print(language_code)



ja_XX


#Translation of the caption into the example language

In [ ]:
#the text or sentence input is "text"

# translate Hindi to French
tokenizer_for_translation.src_lang = "en_XX"

'''
add a new variable which will contain the best captions and add them to the encoder
'''

encoded_lan = tokenizer_for_translation(caption_1, return_tensors="pt")
generated_tokens = model_for_translation.generate(**encoded_lan, forced_bos_token_id=tokenizer_for_translation.lang_code_to_id[language_code])
tokenizer_for_translation.batch_decode(generated_tokens, skip_special_tokens=True)
# => "Le chef de l 'ONU affirme qu 'il n 'y a pas de solution militaire en Syria."


["サンプル2: ['青いメモリードリンで寝ている女の子の画像は、すべての水の中に']"]